In [211]:
import pandas as pd
import numpy as np
import math

In [212]:
#MeanImputer
class MeanImputer():
    def __init__(self, copy=True):
        self.copy = copy
    def __is_numpy(self, X):
        #X : pandas.DataFrame или numpy.ndarray
        #Is it numpy or not
        return isinstance(X, np.ndarray)
    def fit(self, X, y=None):
        self._encoder_dict = {}
        is_np = self.__is_numpy(X)
        #reshape from 1D to 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        #amount of columns
        ncols = X.shape[1]
        
        if is_np:
            for col in range(ncols):
                self._encoder_dict[col] = np.nanmean(X[:, col])
        else:
            for col in X.columns:
                self._encoder_dict[col] = X[col].mean()
                
        return self
    
    def transform(self, X):
        if self.copy:
            X = X.copy()
        is_np = self.__is_numpy(X)
        
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        ncols = X.shape[1]
        
        if is_np:
            for col in range(ncols):
                X[:, col] = np.nan_to_num(
                    X[:, col],
                nan=self._encoder_dict[col])
        else:
            for col in X.columns:
                X[col] = np.where(X[col].isnull(),
                                    self._encoder_dict[col],
                                    X[col])
        return X    

In [213]:
toy_train = pd.DataFrame(
{'Balance': [8.3, np.NaN, 10.2, 3.1],
'Age': [23, 29, 36, np.NaN]})
toy_train

,Balance,Age
0,8.3,23.0
1,NaN,29.0
2,10.2,36.0
3,3.1,NaN


In [214]:
toy_test = pd.DataFrame(
{'Balance': [10.4, np.NaN, 22.5, 1.1],
'Age': [13, 19, 66, np.NaN]})
toy_test

,Balance,Age
0,10.4,13.0
1,NaN,19.0
2,22.5,66.0
3,1.1,NaN


In [215]:
#Using mean()
'''
for col in toy_train.columns:
    toy_train[col].fillna(toy_train[col].mean(), inplace=True)
    toy_test[col].fillna(toy_train[col].mean(), inplace=True)
print('обучающий датафрейм')
print(toy_train)
print('')
print('тестовый датафрейм')
print(toy_test)
'''

"\nfor col in toy_train.columns:\n    toy_train[col].fillna(toy_train[col].mean(), inplace=True)\n    toy_test[col].fillna(toy_train[col].mean(), inplace=True)\nprint('обучающий датафрейм')\nprint(toy_train)\nprint('')\nprint('тестовый датафрейм')\nprint(toy_test)\n"

In [216]:
imp = MeanImputer()
imp.fit(toy_train)
toy_train = imp.transform(toy_train)
toy_train

,Balance,Age
0,8.3,23.000000
1,7.2,29.000000
2,10.2,36.000000
3,3.1,29.333333


In [217]:
toy_test = imp.transform(toy_test)
toy_test

,Balance,Age
0,10.4,13.000000
1,7.2,19.000000
2,22.5,66.000000
3,1.1,29.333333


In [218]:
toy_train = pd.DataFrame(
{'Balance': [8.3, np.NaN, 10.2, 3.1],
'Age': [23, 29, 36, np.NaN]})
# создаем экземпляр класса, отключив копирование
imp = MeanImputer(copy=False)
# обучаем модель
imp.fit(toy_train[['Age']])
# применяем модель
toy_train['Age'] = imp.transform(toy_train[['Age']])
toy_train

C:\Users\Андрей Куликов\AppData\Local\Temp\ipykernel_13088\1614832478.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = np.where(X[col].isnull(),


,Balance,Age
0,8.3,23.000000
1,NaN,29.000000
2,10.2,36.000000
3,3.1,29.333333


In [219]:
np_toy_train = np.array(pd.DataFrame(
{'Balance': [8.3, np.NaN, 10.2, 3.1],
'Age': [23, 29, 36, np.NaN]}))
np_toy_train

np_toy_test = np.array(pd.DataFrame(
{'Balance': [10.4, np.NaN, 22.5, 1.1],
'Age': [13, 19, 66, np.NaN]}))
np_toy_test

array([[10.4, 13. ],
       [ nan, 19. ],
       [22.5, 66. ],
       [ 1.1,  nan]])

In [220]:
imp.fit(np_toy_train)
np_toy_train = imp.transform(np_toy_train)
np_toy_train

array([[ 8.3       , 23.        ],
       [ 7.2       , 29.        ],
       [10.2       , 36.        ],
       [ 3.1       , 29.33333333]])

In [221]:
np_toy_test = imp.transform(np_toy_test)
np_toy_test

array([[10.4       , 13.        ],
       [ 7.2       , 19.        ],
       [22.5       , 66.        ],
       [ 1.1       , 29.33333333]])

In [222]:
#KNN Model code;

class KNN_Estimator():
    def _euclidean_distance(self, x1, x2):
        distance = 0
        for i in range(len(x1)):
            distance += pow((x1[i] - x2[i]), 2)
        return math.sqrt(distance)
    
    def _vote(self, neighbor_labels):
        counts = np.bincount(neighbor_labels.astype('int'))
        return counts.argmax()
    
    def __init__(self, k=5, task='classification'):
        self.k = k
        self.task = task
        self.k_nearest_neighbors_ = []
        
    def fit(self, X, y):
        self.X_memorized = X
        self.y_memorized = y
        
    def predict(self, X):
        y_pred = np.empty(X.shape[0])
        if self.task == 'classification':
            for i, test_sample in enumerate(X):
                idx = np.argsort([self._euclidean_distance(
                    test_sample, x) for x in self.X_memorized])[:self.k]
                k_nearest_neighbors = np.array(
                    [self.y_memorized[i] for i in idx])
                self.k_nearest_neighbors_.append(k_nearest_neighbors)
                y_pred[i] = self._vote(self.k_nearest_neighbors_[i])
        if self.task == 'regression':
            for i, test_sample in enumerate(X):
                idx = np.argsort([self._euclidean_distance(
                    test_sample, x) for x in self.X_memorized])[:self.k]
                k_nearest_neighbors = np.array(
                    [self.y_memorized[i] for i in idx])
                self.k_nearest_neighbors_.append(k_nearest_neighbors)
                y_pred[i] = np.mean(self.k_nearest_neighbors_[i])
        return y_pred    
    

In [223]:
#Classification
X_trn = np.array([[0.1, 0.2, 0.3],
                                [0.7, 0.5, 0.2],
                                [0.1, 0.2, 0.2],
                                [0.9, 0.7, 3.5],
                                [0.2, 0.4, 1.4],
                                [0.4, 0.1, 0.5]])

y_trn = np.array([1, 0, 1, 0, 0, 1])

X_tst = np.array([[0.1, 0.7, 1.1],
                                [0.5, 0.3, 2.8],
                                [0.1, 0.1, 0.2],
                                [0.9, 0.7, 1.5]])

In [224]:
knn = KNN_Estimator(k=3, task='classification')
knn.fit(X_trn, y_trn)

pred = knn.predict(X_tst)
pred

array([1., 0., 1., 0.])

In [225]:
knn.k_nearest_neighbors_

[array([0, 1, 1]), array([0, 0, 1]), array([1, 1, 1]), array([0, 1, 0])]

In [226]:
y_trn = np.array([1.2, 0.5, 1.4, 2.2, 3.5, 5.9])

In [227]:
knn = KNN_Estimator(k=3, task='regression')
knn.fit(X_trn, y_trn)

In [228]:
pred = knn.predict(X_tst)
pred

array([3.53333333, 3.86666667, 2.83333333, 3.3       ])

In [229]:
knn.k_nearest_neighbors_

[array([3.5, 5.9, 1.2]),
 array([2.2, 3.5, 5.9]),
 array([1.4, 1.2, 5.9]),
 array([3.5, 5.9, 0.5])]

In [230]:
a = [2, 4, 7, 9, 14, 20, 21, 22]
b = [3, 5, 8, 10, 14, 20, 21, 30]

def seven(a,b):
    for i in range(len(a)):
            if a[i] % 7 == 0 and b[i] % 7 == 0:
                print(a[i])
            else:
                pass

In [231]:
seven(a,b)

14
21


In [232]:
def find(lst1, lst2):
    lst1 = [i for i in lst1 if i % 7 == 0]
    lst2 = [i for i in lst2 if i % 7 == 0]
    return set(lst1) & set(lst2)

find(a,b)

{14, 21}

In [233]:
a = ["a", "b", "c", "d", "e", "f"]
b = [1, 0, 9, 3, 2, 0]

In [234]:
def get_sorted(lst1, lst2):
    lst_tmp = [x for x in zip(lst1, lst2)]
    lst_tmp.sort(key=lambda x: x[1])
    return [x[0] for x in lst_tmp]

In [235]:
get_sorted(a,b)

['b', 'f', 'a', 'e', 'd', 'c']

In [236]:
z = zip(a,b)
z = list(z)
z

[('a', 1), ('b', 0), ('c', 9), ('d', 3), ('e', 2), ('f', 0)]

In [237]:
#Pipeline

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [238]:
os.getcwd()

'f:\\KULIKOV\\ML\\MLCourseAI\\Data Analytics 2023 V2\\ML DA may 2023'

In [239]:
#Loaddataset
from sklearn.datasets import fetch_covtype
import pandas as pd
#https://scikit-learn.ru/7-2-real-world-datasets/
#https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_outlier_detection_bench.html#sphx-glr-auto-examples-miscellaneous-plot-outlier-detection-bench-py

X, y = fetch_covtype(return_X_y=True, as_frame=True)
s = (y == 2) + (y == 4)
X = X.loc[s]
y = y.loc[s]
y = (y != 2).astype(np.int32)

#X, _, y, _ = train_test_split(X, y, train_size=0.05, stratify=y, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [240]:
#Scaling
standardscaler = StandardScaler()
standardscaler.fit(X_train)
# Transform ONLY to train and test, not to X. Because in X - other meanings.
X_train_standardscaled = standardscaler.transform(X_train)  
X_test_standardscaled = standardscaler.transform(X_test)

In [241]:
#TFitting
logreg = LogisticRegression(solver='lbfgs', max_iter=400)
logreg.fit(X_train_standardscaled, y_train)
print("Score train: {:.3f}".format(
logreg.score(X_train_standardscaled, y_train)))
print("Score test: {:.3f}".format(
logreg.score(X_train_standardscaled, y_train)))

Score train: 1.000
Score test: 1.000


In [242]:
#Getting score
train_score = logreg.score(X_train_standardscaled, y_train)
test_score = logreg.score(X_test_standardscaled, y_test)
print(f"Score train: {train_score:.3f}")
print(f"Score test: {test_score:.3f}")

Score train: 1.000
Score test: 1.000


In [243]:
print("Score train: %.3f" % train_score)
print("Score test: %.3f" % test_score)

Score train: 1.000
Score test: 1.000


In [244]:
#Predictiong
logreg_pred = logreg.predict(X_test_standardscaled)
logreg_pred

array([0, 0, 0, ..., 0, 0, 0])

In [245]:
#Predicting probability
logreg_pred = logreg.predict_proba(X_test_standardscaled)
logreg_pred

array([[9.99999997e-01, 3.43359067e-09],
       [9.99999998e-01, 2.17678902e-09],
       [1.00000000e+00, 1.21786506e-12],
       ...,
       [1.00000000e+00, 2.28650412e-10],
       [1.00000000e+00, 3.19741413e-12],
       [1.00000000e+00, 6.61051203e-19]])

In [246]:
#Constant
intercept = np.round(logreg.intercept_, 3)
intercept

array([-25.609])

In [247]:
#Coef
coef = np.round(logreg.coef_, 3)
coef

array([[-6.863e+00, -4.450e-01,  4.950e-01, -4.370e-01, -3.320e-01,
         4.905e+00,  9.090e-01,  1.671e+00, -1.179e+00,  1.724e+00,
        -2.480e-01,  1.000e-03, -2.350e-01,  1.721e+00,  2.110e-01,
        -4.000e-03,  5.290e-01,  1.100e-02,  1.590e-01,  1.800e-01,
        -1.000e-03, -1.000e-03, -1.000e-03, -1.590e-01,  5.040e-01,
        -1.900e-02, -1.000e-03,  1.890e-01,  0.000e+00, -6.400e-02,
         2.500e-02, -5.600e-02, -0.000e+00, -3.900e-02,  0.000e+00,
         1.000e-03, -0.000e+00, -1.100e-02,  0.000e+00, -4.000e-03,
         1.000e-03,  0.000e+00, -8.100e-02, -2.860e-01,  1.000e-03,
         2.000e-03,  1.000e-03,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00,  1.000e-03,  0.000e+00,  0.000e+00]])

In [248]:
#Classes
classes = np.round(logreg.classes_, 3)
classes

array([0, 1])

In [249]:
logreg.coef_[0]

array([-6.86335327e+00, -4.44872949e-01,  4.95230289e-01, -4.37066814e-01,
       -3.32491131e-01,  4.90456158e+00,  9.09293036e-01,  1.67094079e+00,
       -1.17893584e+00,  1.72418089e+00, -2.47727420e-01,  1.30181700e-03,
       -2.34978485e-01,  1.72075737e+00,  2.11016259e-01, -3.53628838e-03,
        5.28561367e-01,  1.07845255e-02,  1.59423234e-01,  1.79662757e-01,
       -9.17802709e-04, -1.34119874e-03, -7.05269441e-04, -1.59042417e-01,
        5.03849394e-01, -1.86944028e-02, -9.44020346e-04,  1.89178845e-01,
        0.00000000e+00, -6.44847844e-02,  2.51200087e-02, -5.58369297e-02,
       -2.29150022e-04, -3.91221385e-02,  6.21205535e-05,  8.62002215e-04,
       -2.76901069e-04, -1.08280955e-02,  2.09934128e-04, -4.11126506e-03,
        6.19675404e-04,  2.48454983e-04, -8.14092955e-02, -2.86473120e-01,
        5.78602864e-04,  1.72477402e-03,  5.40896563e-04,  3.61707208e-04,
        8.77095533e-05,  1.56332368e-04,  0.00000000e+00,  6.12474037e-04,
        4.17828904e-04,  

In [250]:
#Coef for features
for i,feature in zip(logreg.coef_[0], X_train.columns):
    print(feature, i)

Elevation -6.863353273500244
Aspect -0.44487294882555245
Slope 0.4952302886105166
Horizontal_Distance_To_Hydrology -0.4370668139499791
Vertical_Distance_To_Hydrology -0.3324911313472247
Horizontal_Distance_To_Roadways 4.904561581176035
Hillshade_9am 0.9092930360668068
Hillshade_Noon 1.6709407882924503
Hillshade_3pm -1.1789358445753517
Horizontal_Distance_To_Fire_Points 1.7241808871037037
Wilderness_Area_0 -0.2477274195599548
Wilderness_Area_1 0.0013018169964458474
Wilderness_Area_2 -0.23497848521813966
Wilderness_Area_3 1.7207573721353782
Soil_Type_0 0.2110162589754352
Soil_Type_1 -0.003536288377687285
Soil_Type_2 0.5285613673827958
Soil_Type_3 0.010784525517597626
Soil_Type_4 0.1594232336686379
Soil_Type_5 0.1796627570199406
Soil_Type_6 -0.0009178027089207292
Soil_Type_7 -0.0013411987397579893
Soil_Type_8 -0.0007052694408819774
Soil_Type_9 -0.1590424166118983
Soil_Type_10 0.5038493940734217
Soil_Type_11 -0.018694402803804383
Soil_Type_12 -0.000944020346388549
Soil_Type_13 0.1891788445

In [251]:
#Bootstrap and out-of-bag
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor

In [254]:
X

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2579.0,132.0,6.0,300.0,-15.0,67.0,230.0,237.0,140.0,6031.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,2886.0,151.0,11.0,371.0,26.0,5253.0,234.0,240.0,136.0,4051.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,2742.0,134.0,22.0,150.0,69.0,3215.0,248.0,224.0,92.0,6091.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576882,2617.0,29.0,13.0,390.0,128.0,2081.0,215.0,211.0,130.0,592.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
576883,2614.0,21.0,13.0,379.0,125.0,2051.0,211.0,212.0,135.0,618.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
576884,2612.0,17.0,13.0,371.0,123.0,2021.0,208.0,211.0,138.0,644.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
576885,2610.0,16.0,14.0,365.0,110.0,1991.0,208.0,211.0,138.0,671.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [296]:
#Toy regression

var_lst = ['Aspect', 'Slope',
'Horizontal_Distance_To_Hydrology', 'Elevation']
toy_data = X[var_lst].head(10)
toy_labels = toy_data.pop('Elevation')

In [310]:
toy_data.shape

(10, 3)

In [308]:
#Bootstrap - values can be repeated
sample_indices = np.arange(toy_data.shape[0])
rng = np.random.RandomState(42)
bootstrap_indices = rng.choice(sample_indices,
                               size=sample_indices.shape[0],
                               replace=True)
display(bootstrap_indices.shape)
toy_data_boot = toy_data.iloc[bootstrap_indices]
toy_labels_boot = toy_labels.iloc[bootstrap_indices]
display(toy_data_boot)
display(toy_labels_boot)

(10,)

,Aspect,Slope,Horizontal_Distance_To_Hydrology
27,148.0,16.0,323.0
11,151.0,11.0,371.0
28,135.0,1.0,212.0
12,134.0,22.0,150.0
27,148.0,16.0,323.0
41,346.0,2.0,0.0
5,132.0,6.0,300.0
27,148.0,16.0,323.0
28,135.0,1.0,212.0
12,134.0,22.0,150.0


27    2962.0
11    2886.0
28    2811.0
12    2742.0
27    2962.0
41    2570.0
5     2579.0
27    2962.0
28    2811.0
12    2742.0
Name: Elevation, dtype: float64

In [309]:
toy_labels_boot.nunique()

6

In [311]:
#Out-of-bag:
toy_data_out_boot = toy_data[~toy_data.index.isin(
toy_data_boot.index)]
toy_labels_out_boot = toy_labels[~toy_labels.index.isin(
toy_data_boot.index)]
display(toy_data_out_boot)
display(toy_labels_out_boot)

,Aspect,Slope,Horizontal_Distance_To_Hydrology
2,139.0,9.0,268.0
3,155.0,18.0,242.0
21,209.0,17.0,216.0
35,45.0,19.0,242.0


2     2804.0
3     2785.0
21    2880.0
35    2900.0
Name: Elevation, dtype: float64

In [317]:
#bootstrap and out-of-bag function
def generate_bootstrap(rng, X, y, verbose=True):
        sample_indices = np.arange(X.shape[0])
        bootstrap_indices = rng.choice(sample_indices,
                                       size=sample_indices.shape[0],
                                       replace=True)
        X_boot = X.iloc[bootstrap_indices]
        y_boot = y.iloc[bootstrap_indices]
        X_out_boot = X[~X.index.isin(X_boot.index)]
        y_out_boot = y[~y.index.isin(X_boot.index)]
        if verbose:
            print(f"{i}-iteration")
            print(f"indexes in bootstrap: {X_boot.index.tolist()}")
            print(f"indexes in out-of-bag: {X_out_boot.index.tolist()}\n")
        return X_boot, y_boot, X_out_boot, y_out_boot

In [318]:
rng = np.random.RandomState(42)
standardscaler = StandardScaler()
tree = DecisionTreeRegressor(random_state=42)
test_score_lst = []
for i in range(1, 4):
    X_boot, y_boot, X_out_boot, y_out_boot = generate_bootstrap(
    rng, toy_data, toy_labels)
    tree.fit(X_boot, y_boot)
    test_score = tree.score(X_out_boot, y_out_boot)
    test_score_lst.append(test_score)

1-iteration
indexes in bootstrap: [27, 11, 28, 12, 27, 41, 5, 27, 28, 12]
indexes in out-of-bag: [2, 3, 21, 35]

2-iteration
indexes in bootstrap: [11, 28, 28, 5, 21, 12, 3, 28, 21, 3]
indexes in out-of-bag: [2, 27, 35, 41]

3-iteration
indexes in bootstrap: [12, 2, 41, 21, 35, 2, 41, 5, 27, 11]
indexes in out-of-bag: [3, 28]



In [319]:
print(test_score_lst)

[-9.155198273820488, -0.9437390247197082, -184.94378698224853]


In [320]:
mean_r2 = sum(test_score_lst) / len(test_score_lst)
print("Mean Value R2: %.3f" % mean_r2)

Mean Value R2: -65.014


In [321]:
#R2 in list
import statistics
mean_r2 = statistics.fmean(test_score_lst)
print("Mean Value R2: %.3f" % mean_r2)

Mean Value R2: -65.014


In [324]:

rng = np.random.RandomState(42)

standardscaler = StandardScaler()

logreg = LogisticRegression(solver='lbfgs', max_iter=200)

test_score_lst = []

for i in range(15):
    X_boot, y_boot, X_out_boot, y_out_boot = generate_bootstrap(
        rng, X_train, y_train, verbose=False)
    standardscaler.fit(X_boot)
    X_boot_scaled = standardscaler.transform(X_boot)
    X_out_boot_scaled = standardscaler.transform(X_out_boot)
    logreg.fit(X_boot_scaled, y_boot)
   #     X_out_boot_scaled)
    test_score = logreg.score(
        X_out_boot_scaled, y_out_boot)
    test_score_lst.append(test_score)
    
mean_acc = statistics.fmean(test_score_lst)
print("Mean_acc: %.3f" % mean_acc)

c:\Users\Андрей Куликов\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Андрей Куликов\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s